In [1]:
!pip install azure-datalake-store

  Using cached azure_datalake_store-0.0.48-py2.py3-none-any.whl (53 kB)
  Using cached adal-1.2.3-py2.py3-none-any.whl (53 kB)


In [2]:
from azure.datalake.store import core, lib, multithread


In [3]:
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()


In [6]:
adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FH4CGPXQR to authenticate.


In [7]:
spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])


In [8]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import StringType,StructField, StructType  

In [5]:
df_mes = spark.read.format('csv').option('sep','|').load('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/01_raw/vendas_dia_complemento/mes')

Py4JJavaError: An error occurred while calling o70.load.
: java.io.IOException: Password fs.adl.oauth2.client.id not found
	at org.apache.hadoop.fs.adl.AdlFileSystem.getPasswordString(AdlFileSystem.java:950)
	at org.apache.hadoop.fs.adl.AdlFileSystem.getConfCredentialBasedTokenProvider(AdlFileSystem.java:289)
	at org.apache.hadoop.fs.adl.AdlFileSystem.getAccessTokenProvider(AdlFileSystem.java:268)
	at org.apache.hadoop.fs.adl.AdlFileSystem.initialize(AdlFileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3354)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3403)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3371)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:477)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.datasources.DataSource.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:

In [10]:
df_mes.show()

+----+----+----+-----------+----------+------------+---+-----------+---+----------+----+------------+----------+--------------------+-----+----------+--------------------+-----------------+----+---------+----+-----+------------------+------------------+------------------+------------------+------------------+------------------+-------------+----+------------------+------------------+------------------+------------------+------------------+----------+------------+-------------------+--------+--------+--------------+----+
| _c0| _c1| _c2|        _c3|       _c4|         _c5|_c6|        _c7|_c8|       _c9|_c10|        _c11|      _c12|                _c13| _c14|      _c15|                _c16|             _c17|_c18|     _c19|_c20| _c21|              _c22|              _c23|              _c24|              _c25|              _c26|              _c27|         _c28|_c29|              _c30|              _c31|              _c32|              _c33|              _c34|      _c35|        _c36|       

In [11]:
df_mes = spark.read.format('csv').option('sep','|').schema(schema_mes).load('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/01_raw/vendas_dia_complemento/mes').where("Empr is not null and Empr <> 'Empr' ").select('Data Saida','Cent','Material','IBM','Volume','Canal')

NameError: name 'schema_mes' is not defined

In [87]:
df_mes.show()

+----------+----+----------+----------+-------------------+-----+
|Data Saida|Cent|  Material|       IBM|             Volume|Canal|
+----------+----+----------+----------+-------------------+-----+
|17.02.2020|BARC|24319801  |1033260   |             0,000 |V    |
|17.02.2020|BARC|24144801  |1033260   |             0,000 |V    |
|17.02.2020|BARC|22147801  |1033260   |             0,000 |V    |
|28.02.2020|BARC|24319801  |1033260   |             0,000 |V    |
|28.02.2020|BARC|24144801  |1033260   |             0,000 |V    |
|28.02.2020|BARC|22147801  |1033260   |             0,000 |V    |
|01.02.2020|BARC|24141801  |72931     |        62.000,000 |C    |
|01.02.2020|BARC|22149801  |1033260   |         5.000,000 |V    |
|01.02.2020|BARC|24144801  |1033260   |        25.000,000 |V    |
|01.02.2020|BARC|24314801  |1033260   |        25.000,000 |V    |
|01.02.2020|BARC|24144801  |1033259   |         5.000,000 |V    |
|01.02.2020|BARC|24314801  |1033259   |        35.000,000 |V    |
|01.02.202

In [12]:
schema_mes = StructType([
        StructField('_c0', StringType(), True),
        StructField('Empr', StringType(), True),
        StructField('Cent', StringType(), True),
        StructField('Nota Fiscal', StringType(), True),
        StructField('Data Saida', StringType(), True),
        StructField('Data Criacao', StringType(), True),
        StructField('UF', StringType(), True),
        StructField('Faturamento', StringType(), True),
        StructField('Itm', StringType(), True),
        StructField('Vencimento', StringType(), True),
        StructField('TpOV', StringType(), True),
        StructField('Ordem Vendas', StringType(), True),
        StructField('IBM', StringType(), True),
        StructField('Razao Social', StringType(), True),
        StructField('Canal', StringType(), True),
        StructField('Material', StringType(), True),
        StructField('Descricao', StringType(), True),
        StructField(' Quantidade', StringType(), True),
        StructField('UM', StringType(), True),
        StructField('Cond.Pgto', StringType(), True),
        StructField('Inco', StringType(), True),
        StructField('C.Exp', StringType(), True),
        StructField('Valor item', StringType(), True),
        StructField('Valor ZPFL', StringType(), True),
        StructField('Valor ZPFN', StringType(), True),
        StructField('Valor ZENC', StringType(), True),
        StructField('Imposto', StringType(), True),
        StructField('Valor ZDB1', StringType(), True),
        StructField('Target Block', StringType(), True),
        StructField('UN', StringType(), True),
        StructField('Valor ZDDB', StringType(), True),
        StructField('Valor ZDIA', StringType(), True),
        StructField('Valor ZDFR', StringType(), True),
        StructField('Valor ICM3', StringType(), True),
        StructField('Valor ICS3', StringType(), True),
        StructField('Rep.Vendas', StringType(), True),
        StructField('Acesso ZPFL', StringType(), True),
        StructField('Volume', StringType(), True),
        StructField('UN_', StringType(), True),
        StructField('Hora Fat', StringType(), True),
        StructField('Referência', StringType(), True),
    ])

In [82]:
schema = StructType([
        StructField('Empr', StringType(), True),
        StructField('_c1', StringType(), True),
        StructField('Cent', StringType(), True),
        StructField('Nota Fiscal', StringType(), True),
        StructField('Data Saida', StringType(), True),
        StructField('Data Criacao', StringType(), True),
        StructField('UF', StringType(), True),
        StructField('Faturamento', StringType(), True),
        StructField('Itm', StringType(), True),
        StructField('Vencimento', StringType(), True),
        StructField('TpOV', StringType(), True),
        StructField('Ordem Vendas', StringType(), True),
        StructField('IBM', StringType(), True),
        StructField('Razao Social', StringType(), True),
        StructField('Canal', StringType(), True),
        StructField('Material', StringType(), True),
        StructField('Descricao', StringType(), True),
        StructField(' Quantidade', StringType(), True),
        StructField('UM', StringType(), True),
        StructField('Cond.Pgto', StringType(), True),
        StructField('Inco', StringType(), True),
        StructField('C.Exp', StringType(), True),
        StructField('Valor item', StringType(), True),
        StructField('Valor ZPFL', StringType(), True),
        StructField('Valor ZPFN', StringType(), True),
        StructField('Valor ZENC', StringType(), True),
        StructField('Imposto', StringType(), True),
        StructField('Valor ZDB1', StringType(), True),
        StructField('Target Block', StringType(), True),
        StructField('UN', StringType(), True),
        StructField('Valor ZDDB', StringType(), True),
        StructField('Valor ZDIA', StringType(), True),
        StructField('Valor ZDFR', StringType(), True),
        StructField('Valor ICM3', StringType(), True),
        StructField('Valor ICS3', StringType(), True),
        StructField('Rep.Vendas', StringType(), True),
        StructField('Acesso ZPFL', StringType(), True),
        StructField('Volume', StringType(), True),
        StructField('UN_', StringType(), True),
        StructField('Hora Fat', StringType(), True),
        StructField('Referência', StringType(), True),
    ])

In [47]:
df_dia = spark.read.format('csv').option('sep',';').schema(schema).load('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/01_raw/vendas_dia_complemento/dia').where("Empr is not null and Empr <> 'Empr' ").select('Data Saida','Cent','Material','IBM','Volume','Canal')

In [48]:
df_dia = df_dia.withColumnRenamed('Canal','Segmento')
df_dia.show(5)

+----------+----+--------+-------+---------+--------+
|Data Saida|Cent|Material|    IBM|   Volume|Segmento|
+----------+----+--------+-------+---------+--------+
|09.04.2020|BARC|22149801| 121410|10.000,00|       V|
|09.04.2020|BARC|22149801|1032445| 5.000,00|       V|
|09.04.2020|BARC|22150801|1032445| 5.000,00|       V|
|09.04.2020|BARC|24314801|1032445| 5.000,00|       V|
|09.04.2020|BARC|22149801|1046858|10.000,00|       V|
+----------+----+--------+-------+---------+--------+
only showing top 5 rows



In [49]:
df_dia = df_dia.withColumn('Volume', regexp_replace("Volume", '\.', '')) \
        .withColumn('Volume', regexp_replace("Volume", ',', '.').cast("double")) \
        .withColumn("data_saida", F.to_date(F.col("Data Saida"), "dd.MM.yyyy")).drop("Data Saida")

In [71]:
df_diajoin = df_dia.join(df_vendas_dia, how='outer')

df_diajoin.show()

AnalysisException: 'Detected implicit cartesian product for FULL OUTER join between logical plans\nJoin Inner\n:- Project [Cent#1605, Material#1619, IBM#1615, cast(regexp_replace(regexp_replace(Volume#1641, \\., ), ,, .) as double) AS Volume#1700, Canal#1617, cast(cast(unix_timestamp(Data Saida#1607, dd.MM.yyyy, Some(Etc/UTC)) as timestamp) as date) AS data_saida#1707]\n:  +- Filter (isnotnull(Empr#1603) && NOT (Empr#1603 = Empr))\n:     +- Relation[Empr#1603,_c1#1604,Cent#1605,Nota Fiscal#1606,Data Saida#1607,Data Criacao#1608,UF#1609,Faturamento#1610,Itm#1611,Vencimento#1612,TpOV#1613,Ordem Vendas#1614,IBM#1615,Razao Social#1616,Canal#1617,_c15#1618,Material#1619,Descricao#1620, Quantidade#1621,UM#1622,Cond.Pgto#1623,Inco#1624,C.Exp#1625,Valor item#1626,... 18 more fields] csv\n+- Relation[Date#1733,Base#1734,SKU#1735,IBM#1736,Volume#1737,Segmento#1738] parquet\nand\nRelation[Date#2131,Base#2132,SKU#2133,IBM#2134,Volume#2135,Segmento#2136] parquet\nJoin condition is missing or trivia

In [54]:
df_dia.show()
df_vendas_dia.show()

+----+--------+-------+-------+--------+----------+
|Cent|Material|    IBM| Volume|Segmento|data_saida|
+----+--------+-------+-------+--------+----------+
|BARC|22149801| 121410|10000.0|       V|2020-04-09|
|BARC|22149801|1032445| 5000.0|       V|2020-04-09|
|BARC|22150801|1032445| 5000.0|       V|2020-04-09|
|BARC|24314801|1032445| 5000.0|       V|2020-04-09|
|BARC|22149801|1046858|10000.0|       V|2020-04-09|
|BARC|24144801|1053586| 5000.0|       V|2020-04-09|
|BARC|24314801|1053586|40000.0|       V|2020-04-09|
|BARC|22149801|  33715| 5000.0|       V|2020-04-09|
|BARC|22150801|  33715| 5000.0|       V|2020-04-09|
|BARC|24144801|  33715| 5000.0|       V|2020-04-09|
|BARC|22149801| 120570|10000.0|       V|2020-04-09|
|BARC|22150801| 120570| 5000.0|       V|2020-04-09|
|BARC|22149801|1033260| 5000.0|       V|2020-04-09|
|BARC|24144801|1033260|25000.0|       V|2020-04-09|
|BARC|24314801|1033260|25000.0|       V|2020-04-09|
|BARC|22150801|1032436| 5000.0|       V|2020-04-09|
|BARC|241448

In [94]:
df_dia.alias("a").join(df_vendas_dia.alias("b"), F.col("a.data_saida") == F.col("b.Date").cast("date")).show()

#df_dia.agg(F.min(F.col("data_saida")), F.max(F.col("data_saida"))).show()
#df_vendas_dia.withColumn("foo", F.col("Date").cast("date")).agg(F.min(F.col("foo")), F.max(F.col("foo"))).show()

+----+--------+---+------+--------+----------+----+----+---+---+------+--------+
|Cent|Material|IBM|Volume|Segmento|data_saida|Date|Base|SKU|IBM|Volume|Segmento|
+----+--------+---+------+--------+----------+----+----+---+---+------+--------+
+----+--------+---+------+--------+----------+----+----+---+---+------+--------+



In [51]:
df_vendas_dia = spark.read.parquet('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/03_primary/vendas_dia')

In [53]:
df_vendas_dia.show(5)

+-------------------+----+--------+-------+-------+--------+
|               Date|Base|     SKU|    IBM| Volume|Segmento|
+-------------------+----+--------+-------+-------+--------+
|2018-05-23 00:00:00|APVE|15521801|1004717|  300.0|       A|
|2018-05-23 00:00:00|APVE|15521801|1010270| 1929.0|       A|
|2018-05-23 00:00:00|ARBR|15521801|1010270|  324.0|       A|
|2018-05-23 00:00:00|ARBR|15521801|1014172|19949.0|       A|
|2018-05-23 00:00:00|AREC|15509801|1011068| 1000.0|       A|
+-------------------+----+--------+-------+-------+--------+
only showing top 5 rows



In [73]:
!pip install plotly.express


  Using cached plotly_express-0.4.1-py2.py3-none-any.whl (2.9 kB)
  Using cached plotly-4.6.0-py2.py3-none-any.whl (7.1 MB)
Processing /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf/retrying-1.3.3-py3-none-any.whl


In [72]:
import plotly.express as px
from pyspark.sql import functions as F

ModuleNotFoundError: No module named 'plotly'

In [26]:
vendas_dia_group = df_vendas_dia.groupBy("Mes","Ano","Produto","Segmento").agg({"Volume": "sum"})

AnalysisException: "cannot resolve '`Mes`' given input columns: [Segmento, IBM, Volume, Base, Date, SKU];;\n'Aggregate ['Mes, 'Ano, 'Produto, Segmento#140], ['Mes, 'Ano, 'Produto, Segmento#140, sum(Volume#139) AS sum(Volume)#172]\n+- Relation[Date#135,Base#136,SKU#137,IBM#138,Volume#139,Segmento#140] parquet\n"

In [27]:
vendas_dia_group.show()

NameError: name 'vendas_dia_group' is not defined

In [45]:
v_dia_group_pd = vendas_dia_group.orderBy("Date").toPandas()

In [46]:
px.line(v_dia_group_pd, x="Date", y=["meio", "sum(Volume)"])

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of previous arguments ['Date'] is 3225